<a href="https://colab.research.google.com/github/joanna-regan/CS598_DL4H_StageNet/blob/main/JR_FinalProject_train_ablation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting us to the appropriate directory:

In [1]:
import os
os.chdir('/content/drive/MyDrive/UIUC/CS598_DL4H/Colab_Notebooks/JR_StageNet/StageNet')
os.getcwd()

'/content/drive/MyDrive/UIUC/CS598_DL4H/Colab_Notebooks/JR_StageNet/StageNet'

Initial setups:

In [2]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
from time import perf_counter
import datetime as dt
from datetime import datetime

RANDOM_SEED = 12345
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F

torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

from utils import utils
from utils.readers import DecompensationReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils
from model_ablation1 import StageNet_I

Confirm we're using GPU runtime:

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

available device: cuda:0


Define and load arguments:

In [4]:
def parse_arguments(parser):
    parser.add_argument('--test_mode', type=int, default=0, help='Test SA-CRNN on MIMIC-III dataset')
    parser.add_argument('--data_path', type=str, metavar='<data_path>', help='The path to the MIMIC-III data directory')
    parser.add_argument('--file_name', type=str, metavar='<data_path>', help='File name to save model')
    parser.add_argument('--small_part', type=int, default=0, help='Use part of training data')
    parser.add_argument('--batch_size', type=int, default=128, help='Training batch size')
    parser.add_argument('--epochs', type=int, default=50, help='Training epochs')
    parser.add_argument('--lr', type=float, default=0.001, help='Learing rate')

    parser.add_argument('--input_dim', type=int, default=76, help='Dimension of visit record data')
    parser.add_argument('--rnn_dim', type=int, default=384, help='Dimension of hidden units in RNN')
    parser.add_argument('--output_dim', type=int, default=1, help='Dimension of prediction target')
    parser.add_argument('--dropout_rate', type=float, default=0.5, help='Dropout rate')
    parser.add_argument('--dropconnect_rate', type=float, default=0.5, help='Dropout rate in RNN')
    parser.add_argument('--dropres_rate', type=float, default=0.3, help='Dropout rate in residue connection')
    parser.add_argument('--K', type=int, default=10, help='Value of hyper-parameter K')
    parser.add_argument('--chunk_level', type=int, default=3, help='Value of hyper-parameter K')

    parser.add_argument('-f')

    args = parser.parse_args()
    return args

parser = argparse.ArgumentParser()
args = parse_arguments(parser)
#args = parser.parse_args()

#JR add in my paths:
args.data_path = './data/'
args.file_name = 'trained_model_ablation1'
args.epochs = 50
args.small_part = 5000

Load the train and validation data:

On local device, this took about 12 minutes for train and 1 minute for val.
On Google Colab, should take approxiamtely 2-3 minutes to load both (with small_part = 5000). If running for over 5 minutes, recommend stopping the cell execution and rerunning.

In [8]:
print('Preparing training data ... ')
start_time = perf_counter()

train_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(args.data_path, 'train_subdivided'), 
                                                                   listfile=os.path.join(args.data_path, 'train_listfile.csv'),
                                                                   small_part=args.small_part)
timer1 = perf_counter()
val_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(args.data_path, 'train_subdivided'), 
                                                                 listfile=os.path.join(args.data_path, 'val_listfile.csv'),
                                                                 small_part=args.small_part)
end_time = perf_counter()
        
print("Time to load train data: " + str(dt.timedelta(seconds = timer1 - start_time))) #time to generate the train data loader
print("Time to load validation data: " + str(dt.timedelta(seconds = end_time - timer1))) #time to generate the val data loader

print("Size of training set: " + str(len(train_data_loader._data["X"])))
print("Size of validation set: " + str(len(val_data_loader._data["X"])))

Preparing training data ... 
Generating data...
Generating data...
Time to load train data: 0:02:00.109076
Time to load validation data: 0:01:38.229267
Size of training set: 5000
Size of validation set: 556


In [9]:
#print(len(train_data_loader._data["X"]))
print(len(train_data_loader._data["X"]))

5000


Instantiate Discretizer and Normalizer objects:

In [10]:
discretizer = Discretizer(timestep=1.0, store_masks=True,
                                impute_strategy='previous', start_time='zero')

discretizer_header = discretizer.transform(train_data_loader._data["X"][0])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)
normalizer_state = 'decomp_normalizer'
normalizer_state = os.path.join(os.path.dirname(args.data_path), normalizer_state)
normalizer.load_params(normalizer_state)

Load the Batched Generators:

On local machine, this took about 7 minutes for train and 1 minute for validation.
On Google Colab, should only take 30 seconds for small_part = 5000.

In [11]:
print("Preparing Batched Data Loaders")

start_time = perf_counter()
train_data_gen = utils.BatchGenDeepSupervision(train_data_loader, 
                                               discretizer, normalizer, 
                                               args.batch_size, 
                                               shuffle=True, 
                                               return_names=True)
timer1 = perf_counter()
val_data_gen = utils.BatchGenDeepSupervision(val_data_loader, 
                                             discretizer, 
                                             normalizer, 
                                             args.batch_size, 
                                             shuffle=False, 
                                             return_names=True)
end_time = perf_counter()
        
print("Time to load train generator: " + str(dt.timedelta(seconds = timer1 - start_time))) #time to generate the train data gen
print("Time to load validation generator: " + str(dt.timedelta(seconds = end_time - timer1))) #time to generate the val data gen

Preparing Batched Data Loaders
Time to load train generator: 0:00:24.636538
Time to load validation generator: 0:00:02.821054


Model Construction:

In [12]:
print('Constructing model ... ')
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

model = StageNet_I(args.input_dim+17, args.rnn_dim, args.K, args.output_dim, args.chunk_level, args.dropconnect_rate, args.dropout_rate, args.dropres_rate).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)


Constructing model ... 
available device: cuda:0


Training:

In [13]:
print('Start training ... ')

train_loss = []
val_loss = []
batch_loss = []
max_auprc = 0
val_auroc = []
val_auprc = []
acc = []
prec0 = []
prec1 = []
rec0 = []
rec1 = []
minpse = []

my_epoch_times = []

file_name = './saved_weights/' + args.file_name + datetime.now().strftime("%d%m%Y_%H_%M")
for each_chunk in range(args.epochs):
    epoch_starttime = perf_counter()
    cur_batch_loss = []
    model.train()
    for each_batch in range(train_data_gen.steps):
        batch_data = next(train_data_gen)
        batch_name = batch_data['names']
        batch_data = batch_data['data']

        batch_x = torch.tensor(batch_data[0][0], dtype=torch.float32).to(device)
        batch_mask = torch.tensor(batch_data[0][1], dtype=torch.float32).unsqueeze(-1).to(device)
        batch_y = torch.tensor(batch_data[1], dtype=torch.float32).to(device)
        tmp = torch.zeros(batch_x.size(0),17, dtype=torch.float32).to(device)
        batch_interval = torch.zeros((batch_x.size(0),batch_x.size(1),17), dtype=torch.float32).to(device)
        
        for i in range(batch_x.size(1)):
            cur_ind = batch_x[:,i,-17:]
            tmp+=(cur_ind == 0).float()
            batch_interval[:, i, :] = cur_ind * tmp
            tmp[cur_ind==1] = 0        
        
        if batch_mask.size()[1] > 400:
            batch_x = batch_x[:, :400, :]
            batch_mask = batch_mask[:, :400, :]
            batch_y = batch_y[:, :400, :]
            batch_interval = batch_interval[:, :400, :]

        batch_x = torch.cat((batch_x, batch_interval), dim=-1)
        batch_time = torch.ones((batch_x.size(0), batch_x.size(1)), dtype=torch.float32).to(device)

        optimizer.zero_grad()
        cur_output = model(batch_x, batch_time, device)
        masked_output = cur_output * batch_mask 
        loss = batch_y * torch.log(masked_output + 1e-7) + (1 - batch_y) * torch.log(1 - masked_output + 1e-7)
        loss = torch.sum(loss, dim=1) / torch.sum(batch_mask, dim=1)
        loss = torch.neg(torch.sum(loss))
        cur_batch_loss.append(loss.cpu().detach().numpy())

        loss.backward()
        optimizer.step()
        
        if each_batch % 50 == 0:
            print('Chunk %d, Batch %d: Loss = %.4f'%(each_chunk, each_batch, cur_batch_loss[-1]))

    batch_loss.append(cur_batch_loss)
    train_loss.append(np.mean(np.array(cur_batch_loss)))
    
    epoch_endtime = perf_counter()
    my_epoch_times.append(epoch_endtime - epoch_starttime)

    print('Epoch training time: ' + str(dt.timedelta(seconds = epoch_endtime - epoch_starttime)))
    
    print("\n==>Predicting on validation")
    with torch.no_grad():
        model.eval()
        cur_val_loss = []
        valid_true = []
        valid_pred = []
        for each_batch in range(val_data_gen.steps):
            valid_data = next(val_data_gen)
            valid_name = valid_data['names']
            valid_data = valid_data['data']
            
            valid_x = torch.tensor(valid_data[0][0], dtype=torch.float32).to(device)
            valid_mask = torch.tensor(valid_data[0][1], dtype=torch.float32).unsqueeze(-1).to(device)
            valid_y = torch.tensor(valid_data[1], dtype=torch.float32).to(device)
            tmp = torch.zeros(valid_x.size(0),17, dtype=torch.float32).to(device)
            valid_interval = torch.zeros((valid_x.size(0),valid_x.size(1),17), dtype=torch.float32).to(device)
            
            for i in range(valid_x.size(1)):
                cur_ind = valid_x[:,i,-17:]
                tmp+=(cur_ind == 0).float()
                valid_interval[:, i, :] = cur_ind * tmp
                tmp[cur_ind==1] = 0  
            
            if valid_mask.size()[1] > 400:
                valid_x = valid_x[:, :400, :]
                valid_mask = valid_mask[:, :400, :]
                valid_y = valid_y[:, :400, :]
                valid_interval = valid_interval[:, :400, :]
            
            valid_x = torch.cat((valid_x, valid_interval), dim=-1)
            valid_time = torch.ones((valid_x.size(0), valid_x.size(1)), dtype=torch.float32).to(device)
            
            valid_output = model(valid_x, valid_time, device)
            masked_valid_output = valid_output * valid_mask

            valid_loss = valid_y * torch.log(masked_valid_output + 1e-7) + (1 - valid_y) * torch.log(1 - masked_valid_output + 1e-7)
            valid_loss = torch.sum(valid_loss, dim=1) / torch.sum(valid_mask, dim=1)
            valid_loss = torch.neg(torch.sum(valid_loss))
            cur_val_loss.append(valid_loss.cpu().detach().numpy())

            for m, t, p in zip(valid_mask.cpu().numpy().flatten(), valid_y.cpu().numpy().flatten(), valid_output.cpu().detach().numpy().flatten()):
                if np.equal(m, 1):
                    valid_true.append(t)
                    valid_pred.append(p)

        val_loss.append(np.mean(np.array(cur_val_loss)))
        print('Valid loss = %.4f'%(val_loss[-1]))
        print('\n')
        valid_pred = np.array(valid_pred)
        valid_pred = np.stack([1 - valid_pred, valid_pred], axis=1)
        ret = metrics.print_metrics_binary(valid_true, valid_pred)
        print()

        cur_auprc = ret['auprc']

        val_auprc.append(cur_auprc)
        val_auroc.append(ret['auroc'])
        acc.append(ret['acc'])
        prec0.append(ret['prec0'])
        prec1.append(ret['prec1'])
        rec0.append(ret['rec0'])
        rec1.append(ret['rec1'])
        minpse.append(ret['minpse'])
        
        if cur_auprc > max_auprc:
            max_auprc = cur_auprc
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'chunk': each_chunk
            }
            torch.save(state, file_name)
            print('\n------------ Save best model ------------\n')

Start training ... 
Chunk 0, Batch 0: Loss = 89.8786
Epoch training time: 0:00:53.520867

==>Predicting on validation
Valid loss = 10.8626


confusion matrix:
[[34920   156]
 [  642    72]]
accuracy = 0.9777032732963562
precision class 0 = 0.9819470047950745
precision class 1 = 0.31578946113586426
recall class 0 = 0.9955525398254395
recall class 1 = 0.10084033757448196
AUC of ROC = 0.8454837802380617
AUC of PRC = 0.14940210783254318
min(+P, Se) = 0.22911051212938005


------------ Save best model ------------

Chunk 1, Batch 0: Loss = 14.1058
Epoch training time: 0:00:46.965237

==>Predicting on validation
Valid loss = 10.6967


confusion matrix:
[[34575   501]
 [  557   157]]
accuracy = 0.9704386591911316
precision class 0 = 0.9841455221176147
precision class 1 = 0.23860181868076324
recall class 0 = 0.985716700553894
recall class 1 = 0.21988795697689056
AUC of ROC = 0.8525815332405057
AUC of PRC = 0.15947826372810864
min(+P, Se) = 0.2381615598885794


------------ Save best model ----



---



Evaluate on Test Data

Load last checkpoint:

In [14]:
checkpoint = torch.load(file_name)
save_chunk = checkpoint['chunk']
print("last saved model is in chunk {}".format(save_chunk))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

last saved model is in chunk 21


StageNet_I(
  (kernel): Linear(in_features=94, out_features=1542, bias=True)
  (recurrent_kernel): Linear(in_features=385, out_features=1542, bias=True)
  (nn_scale): Linear(in_features=384, out_features=64, bias=True)
  (nn_rescale): Linear(in_features=64, out_features=384, bias=True)
  (nn_conv): Conv1d(384, 384, kernel_size=(10,), stride=(1,))
  (nn_output): Linear(in_features=384, out_features=1, bias=True)
  (nn_dropconnect): Dropout(p=0.5, inplace=False)
  (nn_dropconnect_r): Dropout(p=0.5, inplace=False)
  (nn_dropout): Dropout(p=0.5, inplace=False)
  (nn_dropres): Dropout(p=0.3, inplace=False)
)

Load test data.

Should take ~1 minute to run. If running for 5+ minutes, stop and rerun the cell.

In [16]:
start_data_loads = perf_counter()

test_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(args.data_path, 'test'), listfile=os.path.join(args.data_path, 'test_listfile.csv'), small_part=args.small_part)
timer1 = perf_counter()

test_data_gen = utils.BatchGenDeepSupervision(test_data_loader, discretizer, normalizer, args.batch_size, shuffle=False, return_names=True)
end_data_loads = perf_counter()

print("Time to load test data: " + str(dt.timedelta(seconds = timer1 - start_data_loads)))
print("Time to load test generator: " + str(dt.timedelta(seconds = end_data_loads - timer1)))

Generating data...
Time to load test data: 0:01:01.894973
Time to load test generator: 0:00:07.014159


Test the model:

In [17]:
print('Testing model ... ')
print('Checkpoint to be loaded: ')
print(file_name)

checkpoint = torch.load(file_name)
save_chunk = checkpoint['chunk']
print("last saved model is in chunk {}".format(save_chunk))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

start_time = perf_counter()
#test_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(args.data_path, 'test'),
#                                                                listfile=os.path.join(args.data_path, 'test_listfile.csv'), small_part=args.small_part)

test_data_loader = common_utils.DeepSupervisionDataLoader(dataset_dir=os.path.join(args.data_path, 'test'),
                                                          listfile=os.path.join(args.data_path, 'test_listfile.csv'),
                                                          small_part=args.small_part)


timer1 = perf_counter()
test_data_gen = utils.BatchGenDeepSupervision(test_data_loader, discretizer,
                                            normalizer, args.batch_size,
                                            shuffle=False, return_names=True)
end_time = perf_counter()

print("Time to load test data: " + str(dt.timedelta(seconds = timer1 - start_time)))
print("Time to load test generator: " + str(dt.timedelta(seconds = end_time - timer1)))
print("Size of test set: " + str(len(test_data_loader._data["X"])))

with torch.no_grad():
    torch.manual_seed(RANDOM_SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(RANDOM_SEED)

    cur_test_loss = []
    test_true = []
    test_pred = []
    
    for each_batch in range(test_data_gen.steps):
        test_data = next(test_data_gen)
        test_name = test_data['names']
        test_data = test_data['data']

        test_x = torch.tensor(test_data[0][0], dtype=torch.float32).to(device)
        test_mask = torch.tensor(test_data[0][1], dtype=torch.float32).unsqueeze(-1).to(device)
        test_y = torch.tensor(test_data[1], dtype=torch.float32).to(device)
        tmp = torch.zeros(test_x.size(0),17, dtype=torch.float32).to(device)
        test_interval = torch.zeros((test_x.size(0),test_x.size(1),17), dtype=torch.float32).to(device)

        for i in range(test_x.size(1)):
            cur_ind = test_x[:,i,-17:]
            tmp+=(cur_ind == 0).float()
            test_interval[:, i, :] = cur_ind * tmp
            tmp[cur_ind==1] = 0  
        
        if test_mask.size()[1] > 400:
            test_x = test_x[:, :400, :]
            test_mask = test_mask[:, :400, :]
            test_y = test_y[:, :400, :]
            test_interval = test_interval[:, :400, :]
        
        test_x = torch.cat((test_x, test_interval), dim=-1)
        test_time = torch.ones((test_x.size(0), test_x.size(1)), dtype=torch.float32).to(device)
        
        test_output = model(test_x, test_time, device)
        masked_test_output = test_output * test_mask

        test_loss = test_y * torch.log(masked_test_output + 1e-7) + (1 - test_y) * torch.log(1 - masked_test_output + 1e-7)
        test_loss = torch.sum(test_loss, dim=1) / torch.sum(test_mask, dim=1)
        test_loss = torch.neg(torch.sum(test_loss))
        cur_test_loss.append(test_loss.cpu().detach().numpy()) 
        
        for m, t, p in zip(test_mask.cpu().numpy().flatten(), test_y.cpu().numpy().flatten(), test_output.cpu().detach().numpy().flatten()):
            if np.equal(m, 1):
                test_true.append(t)
                test_pred.append(p)
    
    print('Test loss = %.4f'%(np.mean(np.array(cur_test_loss))))
    print('\n')
    test_pred = np.array(test_pred)
    test_pred = np.stack([1 - test_pred, test_pred], axis=1)
    test_ret = metrics.print_metrics_binary(test_true, test_pred)

Testing model ... 
Checkpoint to be loaded: 
./saved_weights/trained_model_ablation107052023_18_50
last saved model is in chunk 21
Generating data...
Time to load test data: 0:00:05.284142
Time to load test generator: 0:00:06.992033
Size of test set: 1389
Test loss = 12.9920


confusion matrix:
[[84691   553]
 [ 1155   296]]
accuracy = 0.9802987575531006
precision class 0 = 0.9865456819534302
precision class 1 = 0.34864547848701477
recall class 0 = 0.993512749671936
recall class 1 = 0.20399723947048187
AUC of ROC = 0.8375129247502309
AUC of PRC = 0.20880075444440566
min(+P, Se) = 0.27911784975878706


---

Getting stats from datasets:




In [18]:
import os
import numpy as np
from collections import Counter

my_subject_ids = []
my_length_of_stays = []
test_labels = []
val_labels = []
train_labels = []

for i in range(len(test_data_loader._data["X"])):
    my_subject_ids.append(test_data_loader._data["name"][i].split('_')[0])
    my_length_of_stays.append(test_data_loader._data["ts"][i][-1])
    test_labels.append(test_data_loader._data["ys"][i])
for i in range(len(val_data_loader._data["X"])):
    my_subject_ids.append(val_data_loader._data["name"][i].split('_')[0])
    my_length_of_stays.append(val_data_loader._data["ts"][i][-1])
    val_labels.append(val_data_loader._data["ys"][i])
for i in range(len(train_data_loader._data["X"])):
    my_subject_ids.append(train_data_loader._data["name"][i].split('_')[0])
    my_length_of_stays.append(train_data_loader._data["ts"][i][-1])
    train_labels.append(train_data_loader._data["ys"][i])

In [19]:
print("Total number of ICU stays: " + str(len(my_subject_ids)))
print("Total number of patients: " + str(len(list(set(my_subject_ids)))))
print("--- max number of stays per patient: " + str(max(Counter(my_subject_ids).values())))
print("--- max length of stay: " + str(max(my_length_of_stays)))
#print("--- max length of stay in days: " + str(max(my_length_of_stays)/24))
print("--- min number of stays per patient: " + str(min(Counter(my_subject_ids).values())))
print("--- min length of stay: " + str(min(my_length_of_stays)))
print("--- average number of stays per patient: " + str(sum(Counter(my_subject_ids).values()) / len(Counter(my_subject_ids))))
print("--- average length of stay: " + str(sum(my_length_of_stays) / len(my_length_of_stays)))

Total number of ICU stays: 6945
Total number of patients: 5594
--- max number of stays per patient: 17
--- max length of stay: 2391.0
--- min number of stays per patient: 1
--- min length of stay: 5.0
--- average number of stays per patient: 1.2415087593850553
--- average length of stay: 70.56573074154068


In [20]:
#Breakdowns of train/test/val
total_test_visits = 0
total_pos_test_visits = 0
total_val_visits = 0
total_pos_val_visits = 0
total_train_visits = 0
total_pos_train_visits = 0

for i in range(len(test_labels)):
    test_labels[i] = [int(x) for x in test_labels[i]]
    total_test_visits += len(test_labels[i])
    total_pos_test_visits += sum(test_labels[i])
for i in range(len(val_labels)):
    val_labels[i] = [int(x) for x in val_labels[i]]
    total_val_visits += len(val_labels[i])
    total_pos_val_visits += sum(val_labels[i])
for i in range(len(train_labels)):
    train_labels[i] = [int(x) for x in train_labels[i]]
    total_train_visits += len(train_labels[i])
    total_pos_train_visits += sum(train_labels[i])

total_visits = total_test_visits + total_val_visits + total_train_visits
total_pos_visits = total_pos_test_visits + total_pos_val_visits + total_pos_train_visits

In [21]:
#Total number of visits for *loaded* subsets:
print("Total number of visits (loaded subset): " + str(total_visits) + ", Positive samples: " + str(total_pos_visits))
 
print("--Train--")
print("Number of stays: " + str(len(train_data_loader._data["X"])) + ", Number of visits: " + str(total_train_visits) + ", Number positive visits: " + str(total_pos_train_visits))
print("--Validation--")
print("Number of stays: " + str(len(val_data_loader._data["X"])) + ", Number of visits: " + str(total_val_visits) + ", Number positive visits: " + str(total_pos_val_visits))
print("--Test--")
print("Number of stays: " + str(len(test_data_loader._data["X"])) + ", Number of visits: " + str(total_test_visits) + ", Number positive visits: " + str(total_pos_test_visits))


Total number of visits (loaded subset): 462289, Positive samples: 9544
--Train--
Number of stays: 5000, Number of visits: 329751, Number positive visits: 7307
--Validation--
Number of stays: 556, Number of visits: 38663, Number positive visits: 714
--Test--
Number of stays: 1389, Number of visits: 93875, Number positive visits: 1523


---

Printing some variables to be used for reporting




In [22]:
print(f"args: ", args)
print(f"my_epoch_times: ", my_epoch_times)
print(f"train_loss: ", train_loss)
print(f"val_loss: ", val_loss)
#print(f"batch_loss: ", batch_loss)
print(f"val_auroc: ", val_auroc)
print(f"val_auprc: ", val_auprc)
print(f"acc: ", acc)
print(f"prec0: ", prec0)
print(f"prec1: ", prec1)
print(f"rec0: ", rec0)
print(f"rec1: ", rec1)
print(f"minpse: ", minpse)
print("---")
print(f"max_auprc: ", max_auprc)
print(f"best_epoch: ", save_chunk)

args:  Namespace(test_mode=0, data_path='./data/', file_name='trained_model_ablation1', small_part=5000, batch_size=128, epochs=50, lr=0.001, input_dim=76, rnn_dim=384, output_dim=1, dropout_rate=0.5, dropconnect_rate=0.5, dropres_rate=0.3, K=10, chunk_level=3, f='/root/.local/share/jupyter/runtime/kernel-7e82f817-968f-49e7-aa09-3e2c0aca9165.json')
my_epoch_times:  [53.52086713100016, 46.965236874999846, 46.74542804099974, 47.44141557700004, 47.00352606300021, 46.65724625200028, 47.31189780500017, 46.78698099600024, 47.027830318999804, 47.698594433999915, 47.561601247, 47.58009494299995, 47.102505786000165, 48.13305761999982, 46.67453225100007, 47.73171260399977, 46.70400473400014, 47.63864042400019, 48.47298742099974, 46.77603771200029, 47.07566262799992, 46.59625144499978, 47.57585595799992, 47.782205428, 47.01005685700011, 48.36920047600006, 48.338006080000014, 47.585185436999836, 47.54690949299993, 48.11408615400023, 47.046148511999945, 47.1333141959999, 47.848581878000005, 47.4228